## Felles import for alle kodeblokker

In [2]:
import sys
sys.path.append("../analysis")
from monthly_statistics import MonthlyStatistics
from analyses import DataAnalyzer

import plotly.graph_objects as go
from dash import Dash, html, dcc, Input, Output, callback, no_update
import calendar

analyzer = DataAnalyzer("../../data/processed")
stats = MonthlyStatistics("../../data/processed")

### Enkel visualisering

In [3]:
df_klima = analyzer.langtidsmiddel_per_måned(
    city="tromso",
    element_id="sum(precipitation_amount P1D)",
    statistikk="median",        # evt. "median" eller "std"
)

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=df_klima["month_name"],
        y=df_klima["verdi"],
        name="Oslo"
    )
)
fig.update_layout(
    title="Klimatologisk månedsmiddel for valgt variabel",
    xaxis_title="Måned",
    yaxis_title="Gjennomsnittlig verdi",
    template="plotly_white"
)
fig.show()

## Full Dash app

### Etter komponenten har blitt kjørt, søk opp `127.0.0.1:8050` i nettleser

In [4]:
cities = ["oslo", "tromso"]
element_ids = [
    "mean(air_temperature P1D)",
    "mean(wind_speed P1D)",
    "max(air_temperature P1D)",
    "min(air_temperature P1D)",
    "range(air_temperature P1D)",
    "sum(precipitation_amount P1D)"
]

app = Dash(__name__)
app.title = "Klimatologi med/uten outliers"

app.layout = html.Div(
    style={"maxWidth": "900px", "margin": "auto"},
    children=[
        html.H1("Klimatologisk Månedsmiddel"),

        html.Label("Måletype (elementId):"),
        dcc.Dropdown(
            id="element-dropdown",
            options=[{"label": e, "value": e} for e in element_ids],
            value=element_ids[0],
            clearable=False,
        ),

        html.Label("Statistikk:"),
        dcc.RadioItems(
            id="stat-radio",
            options=[
                {"label": "Middel", "value": "mean"},
                {"label": "Median", "value": "median"},
                {"label": "Std.avvik", "value": "std"},
            ],
            value="mean",
            inline=True,
        ),

        html.Label("Outliers:"),
        dcc.RadioItems(
            id="outlier-radio",
            options=[
                {"label": "Ta med", "value": "med"},
                {"label": "Fjern ekstreme", "value": "uten"},
            ],
            value="med",
            inline=True,
        ),

        html.Label("By(er):"),
        dcc.Checklist(
            id="city-checklist",
            options=[{"label": c.capitalize(), "value": c} for c in cities],
            value=cities,
            inline=True,
        ),

        dcc.Graph(id="climate-graph"),
    ],
)

@app.callback(
    Output("climate-graph", "figure"),
    Input("element-dropdown", "value"),
    Input("stat-radio", "value"),
    Input("outlier-radio", "value"),
    Input("city-checklist", "value"),
)
def update_graph(element_id, statistikk, outlier_choice, selected_cities):

    fig = go.Figure()
    month_order = [calendar.month_abbr[m].capitalize() for m in range(1, 13)]
    remove = (outlier_choice == "uten")

    for city in selected_cities:
        df_klima = analyzer.langtidsmiddel_per_måned(
            city=city,
            element_id=element_id,
            remove_outliers=remove,
            statistikk=statistikk,
        )
        df_klima = df_klima.sort_values("month")  # 1..12

        fig.add_trace(
            go.Bar(
                x=df_klima["month_name"],
                y=df_klima["verdi"],
                name=city.capitalize(),
            )
        )

    t_out = "uten outliers" if remove else "med outliers"
    fig.update_layout(
        title=f"Klimatologisk {statistikk} per måned – {element_id} ({t_out})",
        xaxis_title="Måned",
        yaxis_title="Verdi",
        barmode="group",
        template="plotly_white",
        xaxis=dict(categoryorder="array", categoryarray=month_order),
    )
    return fig


if __name__ == "__main__":
    app.run(debug=True)